In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#weekly0428~0504 原版
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/weekly_0428~0504.html"#每周排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################
weekly_title = []
weekly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(weekly_title_str))#共20名
for i in range(len(weekly_title_str)):
    weekly_title.append(weekly_title_str[i].string)
#print("-"*10)
#print(weekly_title)

####################取得各項餐廳名稱、評級####################
weekly_restaurant = []
weekly_rating = []
weekly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a")
weekly_rating_str = soup.select(".restaurant.item .rating")
#print(weekly_restaurant_str[0].text)
#print(weekly_rating_str[0].text)
for i in range(len(weekly_restaurant_str)//2):
    restaurant = weekly_restaurant_str[i*2].text#這有40格，取一半
    rating = "無紀錄" if(weekly_rating_str[i*2].text=="0")else weekly_rating_str[i*2].text#這有40格，取一半
    weekly_restaurant.append(restaurant)
    weekly_rating.append(rating) 
#print("-"*10)
#print(weekly_restaurant)
#print(weekly_rating)

####################取得各項美食所在區域、產生日期####################
weekly_district = []
weekly_date = []
weekly_date_str = soup.select("h5, .h5")
#print(weekly_date_str)
for i in range(len(weekly_date_str)):
    date_DD_str = weekly_date_str[i].text.split(" ")
    #print(date_DD_str)
    district = date_DD_str[4].replace("(","").replace(")","") if(i==3)else date_DD_str[4].replace("(","").replace(")","") if(i==14)else date_DD_str[4].replace("(","").replace(")","") if(i==18)else date_DD_str[3].replace("(","").replace(")","")
    date = date_DD_str[2].replace("。","") if(i==3)else date_DD_str[2].replace("。","") if(i==14)else date_DD_str[2].replace("。","") if(i==18)else date_DD_str[1].replace("。","")
    weekly_district.append(district)
    weekly_date.append(date)
#print("-"*10)
#print(weekly_district)
#print("-"*10)
#print(weekly_date)

####################取得各項觀看、收藏、推薦次數####################
weekly_watch = []
weekly_collection = []
weekly_recommend = []
weekly_WCR_str = soup.select(".blog-item .stat-box")
#print(weekly_WCR_str)
for i in range(len(weekly_WCR_str)):
    WCR = weekly_WCR_str[i].text.replace("\n","").split(" ")
    weekly_watch.append(WCR[2])
    weekly_collection.append(WCR[4])
    weekly_recommend.append(WCR[6])
#print("-"*10)
#print(weekly_watch)
#print(weekly_collection)
#print(weekly_recommend)

####################取得各項來過人數、食記篇數####################
weekly_visit = []
weekly_foodrecord = []
weekly_VF_str = soup.select(".restaurant.right.item > .ng-binding")
#print(weekly_VF_str)
#print(weekly_VF_str[0].text)
for i in range(len(weekly_VF_str)):
    visit = weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0]
    foodrecord = weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1]
    weekly_visit.append(visit)
    weekly_foodrecord.append(foodrecord)
#print("-"*10)
#print(weekly_visit)
#print(weekly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

weekly_DF = pd.DataFrame(
    {'標題': weekly_title,
     '餐廳名稱': weekly_restaurant,
     '評級': weekly_rating,
     '美食所在區域': weekly_district,
     '產生日期': weekly_date,
     '觀看人數': weekly_watch,
     '收藏人數': weekly_collection,
     '推薦人數': weekly_recommend,
     '多少人來過': weekly_visit,
     '多少篇食記': weekly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

weekly_DF = weekly_DF.rename(index=rows)
print(weekly_DF)
print("-"*10)
weekly_DF_reverse = weekly_DF.T
print(weekly_DF_reverse)
weekly_DF.to_csv('weekly_0428~0504.csv', encoding='utf_8_sig')
weekly_DF_reverse.to_csv('weekly_0428~0504_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題  \
第一名                   王品集團最新品牌！懷舊料理說歷史，經典台菜大集合！丰禾日麗台式小館   
第二名             原創花雕雞 一鍋三種吃法 酒香味濃 真材實料 熱炒也很好吃 喜愛重口味的好選擇   
第三名                Lady nara 台北統一時代店，全球海外首店開幕嘍！信義區新網美餐廳   
第四名                           台北中山 川渝小吃坊 重慶酸辣粉+口水雞開胃爽口    
第五名                       台北美食mumi cafe｜約姐妹一起來烤吐司的韓系咖啡館   
第六名            黃亞細肉骨茶台灣二號店~新加坡正宗肉骨茶名店,白胡椒湯頭香郁,潮州特色滷水美味！   
第七名               台北小旺仔宜蘭蛋餅｜新口感！鋪了滿滿好料的香酥宜蘭蛋餅(景美夜市人氣美食)   
第八名   浮誇又霸氣的料理/超平價無菜單竟然有鮮甜海膽 「東街日本料理」南京復興站/高cp值無菜單料理...   
第九名                   噴汁雞腿可外送，99元大份量漢堡就在IA Captain 國際機長   
第十名               BFF Gossip Brunch-台北公館網美最愛餐廳，姊妹聚會最佳選擇   
第十一名                 烏石水產火鍋超市 自助吧吃到飽59元！火烤兩吃滿額再送哈根達斯冰淇淋   
第十二名                 漂丿燒肉食堂～肉多到你喊怕！豐盛雙拼與震撼戰斧丼飯，還有超值商業午餐   
第十三名       樂越phofun 越式河粉專門，吃一口就被圈粉的超澎湃道地河粉，還可以隨自己喜好加油添醋   
第十四名  福來舖～30餘種中藥材層層調製低溫拌炒，加上大骨燉煮出香醇濃郁、麻而不辣的絕頂湯底！芝山美食...   
第十五名          【新北  港龘港式飲茶】歐式自助百匯(泰山店)。平日中午365元吃到飽/附設停車場   
第十六名           波記香港茶餐廳｜16年老店屹立不搖~老店新開重現經典港式美味｜東區必訪港式茶餐廳 

In [29]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#weekly0505~0511
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/weekly_0505~0511.html"#每周排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################
weekly_title = []
weekly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(weekly_title_str))#共20名
for i in range(len(weekly_title_str)):
    weekly_title.append(weekly_title_str[i].string)
#print("-"*10)
#print(weekly_title)

####################取得各項餐廳名稱、評級####################
weekly_restaurant = []
weekly_rating = []
weekly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a")
weekly_rating_str = soup.select(".restaurant.item .rating")
#print(weekly_restaurant_str[0].text)
#print(weekly_rating_str[0].text) #if(i!=16 or i!=18)else
for i in range(len(weekly_restaurant_str)//2): #16 18 #6 16 17 18
    restaurant = weekly_title_str[16].string.split("．")[1] if(i==16)else weekly_title_str[18].string.split(" ")[0].split("】")[1] if(i==18)else weekly_restaurant_str[i*2].text #這有40格，取一半
    rating = "無紀錄" if(weekly_rating_str[i*2].text=="0")else "無紀錄" if(i==6)else "無紀錄" if(i==16)else "無紀錄" if(i==17)else "無紀錄" if(i==18)else weekly_rating_str[i*2].text#這有40格，取一半
    weekly_restaurant.append(restaurant)
    weekly_rating.append(rating) 
#print("-"*10)
#print(weekly_restaurant)
#print(weekly_rating)

####################取得各項美食所在區域、產生日期####################
weekly_district = []
weekly_date = []
weekly_date_str = soup.select("h5, .h5")
#print(weekly_date_str)
for i in range(len(weekly_date_str)):
    date_DD_str = weekly_date_str[i].text.split(" ")
    #print(date_DD_str) #9 13 14
    district = date_DD_str[4].replace("(","").replace(")","") if(i==9)else date_DD_str[5].replace("(","").replace(")","") if(i==13)else date_DD_str[4].replace("(","").replace(")","") if(i==14)else date_DD_str[3].replace("(","").replace(")","")
    date = date_DD_str[2].replace("。","") if(i==9)else date_DD_str[3].replace("。","") if(i==13)else date_DD_str[2].replace("。","") if(i==14)else date_DD_str[1].replace("。","")
    weekly_district.append(district)
    weekly_date.append(date)
#print("-"*10)
#print(weekly_district)
#print("-"*10)
#print(weekly_date)

####################取得各項觀看、收藏、推薦次數####################
weekly_watch = []
weekly_collection = []
weekly_recommend = []
weekly_WCR_str = soup.select(".blog-item .stat-box")
#print(weekly_WCR_str)
for i in range(len(weekly_WCR_str)):
    WCR = weekly_WCR_str[i].text.replace("\n","").split(" ")
    weekly_watch.append(WCR[2])
    weekly_collection.append(WCR[4])
    weekly_recommend.append(WCR[6])
#print("-"*10)
#print(weekly_watch)
#print(weekly_collection)
#print(weekly_recommend)

####################取得各項來過人數、食記篇數####################
weekly_visit = []
weekly_foodrecord = []
weekly_VF_str = soup.select(".restaurant.right.item > .ng-binding")
#print(weekly_VF_str)
#print(weekly_VF_str[0].text)
for i in range(len(weekly_VF_str)): #16 18
    visit = "無紀錄" if(i==16)else "無紀錄" if(i==18)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0]
    foodrecord = "無紀錄" if(i==16)else "無紀錄" if(i==18)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1]
    weekly_visit.append(visit)
    weekly_foodrecord.append(foodrecord)
#print("-"*10)
#print(weekly_visit)
#print(weekly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

weekly_DF = pd.DataFrame(
    {'標題': weekly_title,
     '餐廳名稱': weekly_restaurant,
     '評級': weekly_rating,
     '美食所在區域': weekly_district,
     '產生日期': weekly_date,
     '觀看人數': weekly_watch,
     '收藏人數': weekly_collection,
     '推薦人數': weekly_recommend,
     '多少人來過': weekly_visit,
     '多少篇食記': weekly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

weekly_DF = weekly_DF.rename(index=rows)
print(weekly_DF)
print("-"*10)
weekly_DF_reverse = weekly_DF.T
print(weekly_DF_reverse)
#weekly_DF.to_csv('weekly_0505~0511.csv', encoding='utf_8_sig')
#weekly_DF_reverse.to_csv('weekly_0505~0511_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題                 餐廳名稱  \
第一名                         小巷內的懷舊日本味，老爺爺的溫馨大阪燒！紅葉日式愛好燒              紅葉日式愛好燒   
第二名       台北信義市府美食:HI5 Cafe好吃夏威夷蝦飯、家常料理，不限時咖啡館、推薦聊天聚餐開會             Hi5 Cafe   
第三名   冠捷黃金脆皮肉圓│饒河夜市少人知道的脆皮肉圓，吃脆皮肉圓不用跑彰化啦！ 饒河夜市美食、台北脆皮肉圓             冠捷黃金脆皮肉圓   
第四名                西門麵店．捷運西門町站｜西門町網紅麵店．蛋包滷肉飯大推．極脆涼麵檸檬香！                 西門麵店   
第五名                       川畝園麵食館 平價好吃選擇多！推薦炒飯、牛肉捲餅、小籠湯包               川畝園麵食館   
第六名   啾哇嘿喲 좋아해요 ｜來自韓文"喜歡你"｜忠孝復興韓式烤肉推薦、超狂韓式烤肉吃到飽，台北必吃...                 啾哇嘿喲   
第七名                        台北第一間～好吃的越南料理！清新店面有冷氣，越南王三重店               越南王三重店   
第八名   延吉街消失十年的人氣麻辣鍋，重現八德路囉！辣上癮極品麻辣鴛鴦鍋,每人鍋底120元,鴨血豆腐無...         辣上癮 極品麻辣鴛鴦火鍋   
第九名                         道地日本濃厚味三上咖哩超人氣咖哩飯炸雞啤酒日等你來挑戰                 三上咖哩   
第十名                              胖肚肚平價燒烤吃到飽，肋眼、天使紅蝦是哪招？             胖肚肚燒肉吃到飽   
第十一名                                 包好吃碳烤鹽酥雞，姊妹聚會必點的美食                碳烤包好吃   
第十二名                化妝台座位太可愛～咖啡館還有賣辣牛奶泡菜

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#weekly0512~0518
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/weekly_0512~0518.html"#每周排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################
weekly_title = []
weekly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(weekly_title_str))#共20名
for i in range(len(weekly_title_str)):
    weekly_title.append(weekly_title_str[i].string)
#print("-"*10)
#print(weekly_title)

####################取得各項餐廳名稱、評級####################
weekly_restaurant = []
weekly_rating = []
weekly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a")#11 16
weekly_rating_str = soup.select(".restaurant.item .rating")#2 5 11 16 18
#print(weekly_restaurant_str[0].text)
#print(weekly_rating_str[0].text)
for i in range(len(weekly_restaurant_str)//2):
    restaurant = weekly_title_str[11].string.split("(")[1].replace(")","") if(i==11)else weekly_title_str[16].string.split(" ")[1].split("-")[0] if(i==16)else weekly_restaurant_str[i*2].text #這有40格，取一半
    rating = "無紀錄" if(weekly_rating_str[i*2].text=="0")else "無紀錄" if(i==2)else "無紀錄" if(i==5)else"無紀錄" if(i==11)else"無紀錄" if(i==16)else "無紀錄" if(i==18)else weekly_rating_str[i*2].text#這有40格，取一半
    weekly_restaurant.append(restaurant)
    weekly_rating.append(rating) 
#print("-"*10)
#print(weekly_restaurant)
#print(weekly_rating)

####################取得各項美食所在區域、產生日期####################
weekly_district = []
weekly_date = []
weekly_date_str = soup.select("h5, .h5")
#print(weekly_date_str)
for i in range(len(weekly_date_str)):
    date_DD_str = weekly_date_str[i].text.split(" ")
    #print(date_DD_str)
    district = date_DD_str[4].replace("(","").replace(")","") if(i==1)else date_DD_str[3].replace("(","").replace(")","")
    date = date_DD_str[2].replace("。","") if(i==1)else date_DD_str[1].replace("。","")
    weekly_district.append(district)
    weekly_date.append(date)
#print("-"*10)
#print(weekly_district)
#print("-"*10)
#print(weekly_date)

####################取得各項觀看、收藏、推薦次數####################
weekly_watch = []
weekly_collection = []
weekly_recommend = []
weekly_WCR_str = soup.select(".blog-item .stat-box")
#print(weekly_WCR_str)
for i in range(len(weekly_WCR_str)):
    WCR = weekly_WCR_str[i].text.replace("\n","").split(" ")
    weekly_watch.append(WCR[2])
    weekly_collection.append(WCR[4])
    weekly_recommend.append(WCR[6])
#print("-"*10)
#print(weekly_watch)
#print(weekly_collection)
#print(weekly_recommend)

####################取得各項來過人數、食記篇數####################
weekly_visit = []
weekly_foodrecord = []
weekly_VF_str = soup.select(".restaurant.right.item > .ng-binding")#11 16
#print(weekly_VF_str)
#print(weekly_VF_str[0].text)
for i in range(len(weekly_VF_str)):
    visit = "無紀錄" if(i==11)else "無紀錄" if(i==16)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0]
    foodrecord = "無紀錄" if(i==11)else "無紀錄" if(i==16)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1]
    weekly_visit.append(visit)
    weekly_foodrecord.append(foodrecord)
#print("-"*10)
#print(weekly_visit)
#print(weekly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

weekly_DF = pd.DataFrame(
    {'標題': weekly_title,
     '餐廳名稱': weekly_restaurant,
     '評級': weekly_rating,
     '美食所在區域': weekly_district,
     '產生日期': weekly_date,
     '觀看人數': weekly_watch,
     '收藏人數': weekly_collection,
     '推薦人數': weekly_recommend,
     '多少人來過': weekly_visit,
     '多少篇食記': weekly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

weekly_DF = weekly_DF.rename(index=rows)
print(weekly_DF)
print("-"*10)
weekly_DF_reverse = weekly_DF.T
print(weekly_DF_reverse)
weekly_DF.to_csv('weekly_0512~0518.csv', encoding='utf_8_sig')
weekly_DF_reverse.to_csv('weekly_0512~0518_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題  \
第一名     IG火紅冰品，來自日本的膠原蛋白牛奶冰，吸睛又美味．大とろ牛乳Ootoromilk (台北店)   
第二名              終於吃到必吃九宮格！搬到市區超方便！人氣依舊滿滿超好拍「吾獨食驗室」（菜單）   
第三名   IKEA新店店｜玻璃屋景觀餐廳、IKEA CAFÉ、挑高設計商場、與捷運相連，新店最美的IK...   
第四名                     雞窩餐廳-人少也能吃驥園砂鍋土雞湯,台北信義安和站中式聚餐推薦   
第五名           爆漿芋泥肉鬆起酥麵包，超ㄉㄨㄞ的芋頭水晶蛋糕，厚實的芋泥餡難以招架．土司吐司烘焙坊   
第六名          金澤咖哩不用飛到北陸也吃的到，日本人開的咖哩專賣店，三重私房美食又添一．No.1咖哩   
第七名                     泰蝦PA水道泰國蝦吃到飽/蝦控必訪水道流水泰國蝦無限吃到飽餐廳   
第八名          台北Sweet Josun 誠品南西店｜一個人的韓式炸雞這裡吃(韓國品牌炸雞速食店)   
第九名            食鮮日式火鍋 399元吃到飽還免服務費! 現做手卷、肉品，還有隱藏特製味噌魚頭湯   
第十名                   起司控與芋泥控都可以！早餐店的手工蛋餅，中正紀念堂附近早餐店～早澤   
第十一名                                雷記燒臘，七寶飯超浮誇、招牌飯最受歡迎   
第十二名  藏在二樓，連在地人都不知道的好吃咖哩和紅酒牛肋飯！大推薦～再訪度百分之百！小說食堂重新店(雙...   
第十三名                     在地人才知道的好吃手工蛋餅，半熟蛋蛋餅是隱藏版點法．百屯豆漿   
第十四名         老椰兄新泰式料理│大直ATT泰好吃，南洋泰式新風味！ 大直att美食、大直att餐廳   
第十五名                         天母小吃｜好食在．超實在的麵店手工蘿蔔糕更是不能錯過   
第十六名                 綻1852 牛排館，牛排龍蝦專門店 (前雅室牛排主廚) (文末菜單) 

In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#weekly0519~0525
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/weekly_0519~0525.html"#每周排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################
weekly_title = []
weekly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(weekly_title_str))#共20名
for i in range(len(weekly_title_str)):
    weekly_title.append(weekly_title_str[i].string)
#print("-"*10)
#print(weekly_title)

####################取得各項餐廳名稱、評級####################
weekly_restaurant = []
weekly_rating = []
weekly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a")#6
weekly_rating_str = soup.select(".restaurant.item .rating")#6 15
#print(weekly_restaurant_str[0].text)
#print(weekly_rating_str[0].text)
for i in range(len(weekly_restaurant_str)//2):
    restaurant = weekly_restaurant_str[i*2].text if(i!=6)else weekly_title_str[6].string.split("．")[1] #這有40格，取一半
    rating = "無紀錄" if(weekly_rating_str[i*2].text=="0")else "無紀錄" if(i==6)else "無紀錄" if(i==15)else weekly_rating_str[i*2].text#這有40格，取一半
    weekly_restaurant.append(restaurant)
    weekly_rating.append(rating)
#print("-"*10)
#print(weekly_restaurant)
#print(weekly_rating)

####################取得各項美食所在區域、產生日期####################
weekly_district = []
weekly_date = []
weekly_date_str = soup.select("h5, .h5")
#print(weekly_date_str)
for i in range(len(weekly_date_str)):#11：24、15：46
    date_DD_str = weekly_date_str[i].text.split(" ")
    #print(date_DD_str)
    district = date_DD_str[4].replace("(","").replace(")","") if(i==11)else date_DD_str[6].replace("(","").replace(")","") if(i==15)else date_DD_str[3].replace("(","").replace(")","")
    date = date_DD_str[2].replace("。","") if(i==11)else date_DD_str[4].replace("。","") if(i==15)else date_DD_str[1].replace("。","")
    weekly_district.append(district)
    weekly_date.append(date)
#print("-"*10)
#print(weekly_district)
#print("-"*10)
#print(weekly_date)

####################取得各項觀看、收藏、推薦次數####################
weekly_watch = []
weekly_collection = []
weekly_recommend = []
weekly_WCR_str = soup.select(".blog-item .stat-box")
#print(weekly_WCR_str)
for i in range(len(weekly_WCR_str)):
    WCR = weekly_WCR_str[i].text.replace("\n","").split(" ")
    weekly_watch.append(WCR[2])
    weekly_collection.append(WCR[4])
    weekly_recommend.append(WCR[6])
#print("-"*10)
#print(weekly_watch)
#print(weekly_collection)
#print(weekly_recommend)

####################取得各項來過人數、食記篇數####################
weekly_visit = []
weekly_foodrecord = []
weekly_VF_str = soup.select(".restaurant.right.item > .ng-binding")
#print(weekly_VF_str)#6
#print(weekly_VF_str[0].text)#6
for i in range(len(weekly_VF_str)):
    visit = "無紀錄" if(i==6)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0]
    foodrecord = "無紀錄" if(i==6)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1]
    weekly_visit.append(visit)
    weekly_foodrecord.append(foodrecord)
#print("-"*10)
#print(weekly_visit)
#print(weekly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

weekly_DF = pd.DataFrame(
    {'標題': weekly_title,
     '餐廳名稱': weekly_restaurant,
     '評級': weekly_rating,
     '美食所在區域': weekly_district,
     '產生日期': weekly_date,
     '觀看人數': weekly_watch,
     '收藏人數': weekly_collection,
     '推薦人數': weekly_recommend,
     '多少人來過': weekly_visit,
     '多少篇食記': weekly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

weekly_DF = weekly_DF.rename(index=rows)
print(weekly_DF)
print("-"*10)
weekly_DF_reverse = weekly_DF.T
print(weekly_DF_reverse)
weekly_DF.to_csv('weekly_0519~0525.csv', encoding='utf_8_sig')
weekly_DF_reverse.to_csv('weekly_0519~0525_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題  \
第一名   大阪爆紅的生吐司在台灣也吃得到囉！超濕潤的口感，吐司邊毫無存在感．Share le Pain...   
第二名      台北中山:美天餐室 90年代舊香港茶餐廳，聽劉德華吃西多士、金山醬烤雞腿，再喝杯鴛鴦絲襪奶茶   
第三名      好食多涮涮屋南西店新開幕，超大肉盤始祖、開幕優惠活龍蝦一隻僅 $300超划算😍 (文末菜單)   
第四名          在地人都電話預訂隱藏版排隊雞排，豆乳鹽酥雞老字號！ 西門町雞排、180c古早味鹽酥雞   
第五名                      浮誇系海鮮麵線，小卷鮮蚵大腸五種配料一次塞爆麵線!! 面線町   
第六名                                  福德涼麵│24小時營業│人氣排隊名店   
第七名             滷味的價格、鐵板燒的食材，雞睪丸、牛鞭、澎湖小卷、日本干貝通通都能滷．美鳳滷味   
第八名     台北咖哩飯新開幕-CURRY STAND 1983,好吃日式乾咖哩＆燉牛肉,中山國中站美食推薦   
第九名                  台北人的宵夜和早餐！越晚越多人～劉媽媽涼麵，蒜味濃郁好吃，邪惡半熟蛋   
第十名            配菜豐富物有所值的日式定食-芝生食堂(忠孝新生店)@捷運忠孝新生站@台北科技大學   
第十一名  傑克兄弟牛排館 臺北信義店 Jack Brothers Steakhouse Taipei｜...   
第十二名  何家古制秘糖 甜而不膩的冰品 IG熱門花生豬血糕冰 珍珠奶茶雪花冰/珍想芋見泥 板橋必吃美食...   
第十三名             台北東區忠孝復興宵夜聚餐:十二月粥品·茶飲·私房菜大安店,也有一人份的砂鍋粥   
第十四名         揚日式料理(生魚片蓋飯),網友大推超威的生魚丼飯,客製化丼飯,中和巷弄美食,無法訂位   
第十五名               金香豬腳。涼麵＿豪邁的將滷大腸一口塞進嘴巴好過癮＿膠質滿滿的滷肉飯很可以   
第十六名  台北大安｜菊乃屋kikunoya～質感無印風日式自助食堂！東京咖哩姊妹店的美味定食料理（捷運... 

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#weekly0526~0601
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/weekly_0526~0601.html"#每周排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################
weekly_title = []
weekly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(weekly_title_str))#共20名
for i in range(len(weekly_title_str)):
    weekly_title.append(weekly_title_str[i].string)
#print("-"*10)
#print(weekly_title)

####################取得各項餐廳名稱、評級####################
weekly_restaurant = []
weekly_rating = []
weekly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a") #1,14
weekly_rating_str = soup.select(".restaurant.item .rating") #1,14
#print(weekly_restaurant_str[0].text)
#print(weekly_rating_str[0].text)
for i in range(len(weekly_restaurant_str)//2):
    restaurant = weekly_title_str[1].string.split("｜")[0] if(i==1)else weekly_title_str[14].string.split("：")[1].split("｜")[0] if(i==14)else weekly_restaurant_str[i*2].text#這有40格，取一半
    rating = "無紀錄" if(weekly_rating_str[i*2].text=="0")else "無紀錄" if(i==1)else "無紀錄" if(i==14)else weekly_rating_str[i*2].text#這有40格，取一半
    weekly_restaurant.append(restaurant)
    weekly_rating.append(rating)
#print("-"*10)
#print(weekly_restaurant)
#print(weekly_rating)

####################取得各項美食所在區域、產生日期####################
weekly_district = []
weekly_date = []
weekly_date_str = soup.select("h5, .h5")
#print(weekly_date_str)
for i in range(len(weekly_date_str)): #7,19
    date_DD_str = weekly_date_str[i].text.split(" ")
    #print(date_DD_str)
    district = date_DD_str[4].replace("(","").replace(")","") if(i==7)else date_DD_str[3].replace("(","").replace(")","")
    date = date_DD_str[2].replace("。","") if(i==7)else date_DD_str[1].replace("。","")
    weekly_district.append(district)
    weekly_date.append(date)
#print("-"*10)
#print(weekly_district)
#print("-"*10)
#print(weekly_date)

####################取得各項觀看、收藏、推薦次數####################
weekly_watch = []
weekly_collection = []
weekly_recommend = []
weekly_WCR_str = soup.select(".blog-item .stat-box")
#print(weekly_WCR_str)
for i in range(len(weekly_WCR_str)):
    WCR = weekly_WCR_str[i].text.replace("\n","").split(" ")
    weekly_watch.append(WCR[2])
    weekly_collection.append(WCR[4])
    weekly_recommend.append(WCR[6])
#print("-"*10)
#print(weekly_watch)
#print(weekly_collection)
#print(weekly_recommend)

####################取得各項來過人數、食記篇數####################
weekly_visit = []
weekly_foodrecord = []
weekly_VF_str = soup.select(".restaurant.right.item > .ng-binding")
#print(weekly_VF_str) #1,14
#print(weekly_VF_str[0].text) #1,14
for i in range(len(weekly_VF_str)):
    visit = "無紀錄" if(i==1)else "無紀錄" if(i==14)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0]
    foodrecord = "無紀錄" if(i==1)else "無紀錄" if(i==14)else weekly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1]
    weekly_visit.append(visit)
    weekly_foodrecord.append(foodrecord)
#print("-"*10)
#print(weekly_visit)
#print(weekly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

weekly_DF = pd.DataFrame(
    {'標題': weekly_title,
     '餐廳名稱': weekly_restaurant,
     '評級': weekly_rating,
     '美食所在區域': weekly_district,
     '產生日期': weekly_date,
     '觀看人數': weekly_watch,
     '收藏人數': weekly_collection,
     '推薦人數': weekly_recommend,
     '多少人來過': weekly_visit,
     '多少篇食記': weekly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

weekly_DF = weekly_DF.rename(index=rows)
print(weekly_DF)
print("-"*10)
weekly_DF_reverse = weekly_DF.T
print(weekly_DF_reverse)
weekly_DF.to_csv('weekly_0526~0601.csv', encoding='utf_8_sig')
weekly_DF_reverse.to_csv('weekly_0526~0601_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題  \
第一名   邱太太甜品坊,爆流的芋泥土石泥,台北人氣芋頭冰,點上一碗二個人吃剛剛好,下午茶點心,外帶,外...   
第二名                         孔明堂｜來這只買草莓大福和麵包？隱藏版蛋黃酥才是重點！   
第三名           板橋美食:星馬廚房 色澤誘人的海南雞飯、燒雞腿飯、椒麻豬排，高人氣在地平價星馬料理   
第四名               咖啡+按摩只要199元！全台首間睡眠咖啡館，快速充電補給站．小憩睡眠咖啡館   
第五名                                 鮨 松濤，捷運劍潭站神祕低調江戶前壽司   
第六名                台北陳耀訓・麵包埠Yoshi Bakery｜世界麵包冠軍北上開店話題十足   
第七名   台北早午餐推薦！好拍小清新早餐店，手桿蛋餅、本丸有創意！沐沐家朝食（氵木氵木 · 家）MuM...   
第八名                                                裸湯拉麵   
第九名                       珍好呷越南河粉｜藏身士東市場份量大到連國父都會怕的越南美食   
第十名               古亭站美食，隱藏版巷弄美食，日本爺爺掌廚的日式炒麵、愛好燒．紅葉日式愛好燒   
第十一名                               享鴨｜王品烤鴨品牌｜兩人也能吃｜東區美食   
第十二名                        小凱悅南村小吃店 四四南村巷弄眷村味，平價好吃炒麵滷味   
第十三名                公館美食，必吃酥香的冰火菠蘿油以及秘制絲襪奶茶．何太守港式菠蘿包專賣店   
第十四名            玖佰號烤豬五花、火鍋，兩百元就能吃平價版七色五花豬烤肉，萬華區美食(姊姊食記)   
第十五名       台北早餐：mornings 流浪早餐車｜限定三個半小時的早餐時間，與鐵板三明治說聲早安！   
第十六名                        台北合菜推薦 :: 金山客家小館 藏不住香氣的客家菜！ 